In [ ]:
from FiinQuantX import FiinSession
# Đăng nhập
username = 'DSTC_36@fiinquant.vn'
password = 'Fiinquant0606'

client = FiinSession(username=username, password=password).login()
fi = client.FiinIndicator()

In [3]:
hose = list(client.TickerList(ticker="VNINDEX"))
print(hose)

['CCC', 'SBG', 'FUCTVGF3', 'FUEIP100', 'GMH', 'FUEKIV30', 'NO1', 'FUCTVGF4', 'RYG', 'FUEDCMID', 'FUEKIVFS', 'FUEMAVND', 'FUEFCV50', 'FUEBFVND', 'FUCTVGF5', 'FUEKIVND', 'FUEABVND', 'FUETCC50', 'FUETPVND', 'AAA', 'AAM', 'AAT', 'ABR', 'ABS', 'ABT', 'ACB', 'ACC', 'ACL', 'ADP', 'AGR', 'AGG', 'ACG', 'ATP', 'ANV', 'APG', 'APH', 'HII', 'ASG', 'ASM', 'ASP', 'BAF', 'BBC', 'BCE', 'BCG', 'BFC', 'BHN', 'BIC', 'BID', 'BCM', 'DBD', 'BWE', 'BKG', 'BMC', 'BMI', 'BMP', 'BRC', 'BSI', 'BSR', 'BTP', 'BTT', 'BVH', 'TNH', 'C32', 'C47', 'CCI', 'CCL', 'CDC', 'CRE', 'STK', 'CHP', 'CIG', 'CII', 'CKG', 'CLC', 'ADG', 'CLL', 'CLW', 'CMG', 'CMV', 'CMX', 'CNG', 'COM', 'CRC', 'CSM', 'CSV', 'CTD', 'CTF', 'CTG', 'CTI', 'ICT', 'CTR', 'CTS', 'CVT', 'D2D', 'DAH', 'ADS', 'DPG', 'DBC', 'DBT', 'DC4', 'DCL', 'DCM', 'DGC', 'DGW', 'DHA', 'DHC', 'DHG', 'DHM', 'TTE', 'DIG', 'DLG', 'DMC', 'DSC', 'DSE', 'DPM', 'DPR', 'DQC', 'DRC', 'DRH', 'DRL', 'DSN', 'DTA', 'DTL', 'DTT', 'DVP', 'DXG', 'DXS', 'DXV', 'FUESSV50', 'E1VFVN30', 'EIB', 'E

In [4]:
from datetime import datetime, timedelta
hose = list(client.TickerList(ticker="VNINDEX"))
hnx = list(client.TickerList(ticker="HNXINDEX"))
upcom = list(client.TickerList(ticker="UPCOMINDEX"))
all_tickers = hose + hnx + upcom
print(len(all_tickers))



1653


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
batch_size = 20
ticker_batches = [all_tickers[i:i + batch_size] for i in range(0, len(all_tickers), batch_size)]
from_date = (datetime.today() - timedelta(days=365)).strftime("%Y-%m-%d")
to_date = datetime.today().strftime("%Y-%m-%d")

# 5. Hàm fetch từng batch
def fetch_batch(batch):
    try:
        return client.Fetch_Trading_Data(
            realtime=False,
            tickers=batch,
            fields=['open','high','low','close','volume','bu','sd','fn','fs','fb'],
            adjusted=True,
            by="1d",
            from_date= '2023-01-01',
            to_date=to_date
        ).get_data()
    except Exception as e:
        print(f"Lỗi batch {batch}: {e}")
        # return pd.DataFrame()

# 6. Chạy song song
all_data = []
with ThreadPoolExecutor(max_workers=10) as executor:  # có thể tăng >10 nếu máy mạnh
    futures = [executor.submit(fetch_batch, b) for b in ticker_batches]
    for future in as_completed(futures):
        all_data.append(future.result())

# 7. Gộp kết quả
final_df = pd.concat(all_data, ignore_index=True)
print(final_df)

Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a while. Please wait...
Fetching data, it may take a wh

In [7]:
print(final_df)

        ticker         timestamp      open      high       low     close  \
0          AAM  2023-01-03 00:00  11284.80  11284.80  11002.68  11190.76   
1          AAM  2023-01-04 00:00  10673.54  11002.68  10673.54  10908.64   
2          AAM  2023-01-05 00:00  10438.44  11190.76  10438.44  11190.76   
3          AAM  2023-01-06 00:00  11002.68  11143.74  10955.66  10955.66   
4          AAM  2023-01-09 00:00  11284.80  11284.80  10908.64  10955.66   
...        ...               ...       ...       ...       ...       ...   
1040146    VXT  2025-08-25 00:00  11000.00  11000.00  11000.00  11000.00   
1040147    VXT  2025-08-26 00:00  11000.00  11000.00  11000.00  11000.00   
1040148    VXT  2025-08-27 00:00  11000.00  11000.00  11000.00  11000.00   
1040149    VXT  2025-08-28 00:00  11000.00  11000.00  11000.00  11000.00   
1040150    VXT  2025-08-29 00:00  11000.00  11000.00  11000.00  11000.00   

         volume      bu      sd         fn         fs         fb  
0        3601.0  290

In [8]:
# Kiểm tra số lượng hàng toàn null
print("Số hàng toàn null:", final_df.isnull().all(axis=1).sum())
# Hiển thị các hàng toàn null
print(final_df[final_df.isnull().all(axis=1)])


Số hàng toàn null: 0
Empty DataFrame
Columns: [ticker, timestamp, open, high, low, close, volume, bu, sd, fn, fs, fb]
Index: []


In [9]:
final_df.to_csv("all_stock_data.csv", index=False)